In [13]:
import pandas as pd
import glob
import json

path = './data/albam'
files = glob.glob(path + "/*.csv")
li = []

for file in files:
    if file != './data/albam/total.csv':
        df = pd.read_csv(file, sep=',', escapechar='\n')
        li.append(df)

albam = pd.concat(li, axis=0, ignore_index=True)
albam.head()

,이름,시급,연장근무,야간근무,휴일근무,근무인정시간,총급여,기본급여,주휴수당
0,김정건(688307),8590,NaN,NaN,NaN,2시간1분,17323,17323.0,0.0
1,김정건(688307),8590,NaN,NaN,NaN,8시간58분,77024,77024.0,0.0
2,백정민(690125),8590,NaN,NaN,NaN,-,0,0.0,0.0
3,백정민(690125),8590,NaN,NaN,NaN,4시간43분,40516,40516.0,0.0
4,백정민(690125),8590,NaN,NaN,NaN,4시간21분,37367,37367.0,0.0


In [14]:
computed = pd.DataFrame()
computed['이름'] = albam['이름'].apply(lambda row: row[:row.find('(')])
computed['시급'] = albam['시급'].astype(int)
computed['근무hrs'] = albam['근무인정시간'].apply(lambda row: row[:row.find('시간')] if row != '-' else 0).astype(int)
computed['근무mins'] = albam['근무인정시간'].apply(lambda row: row[row.find('간')+1:row.find('분')] if row != '-' else 0).astype(int)
computed['총급여'] = albam['총급여'].astype(float)
computed['주휴수당'] = albam['주휴수당']

computed

,이름,시급,근무hrs,근무mins,총급여,주휴수당
0,김정건,8590,2,1,17323.0,0.0
1,김정건,8590,8,58,77024.0,0.0
2,백정민,8590,0,0,0.0,0.0
3,백정민,8590,4,43,40516.0,0.0
4,백정민,8590,4,21,37367.0,0.0
...,...,...,...,...,...,...
83,이웅,8590,5,9,44239.0,0.0
84,김정건,8590,8,58,77024.0,0.0
85,백정민,8590,4,43,40516.0,0.0
86,신수정,8590,10,37,91197.0,0.0


In [15]:
df = pd.DataFrame()
df['이름'] = computed['이름'].unique()
df['세전급여'] = 0
df['주휴수당'] = 0
df['4대보험'] = 0
df['세후급여'] = 0

for idx, row in computed.iterrows():
    name = row['이름']
    joohue = row['주휴수당']
    hrs = computed[computed['이름'] == name]['근무hrs'].sum()
    mins = computed[computed['이름'] == name]['근무mins'].sum()
    if mins > 60:
        hrs += 1
        mins -= 60

    gross_salary = row['총급여']
    insurance = gross_salary * 0.009628

    df.loc[df['이름'] == name, '일한시간'] = f"{hrs}시간 {mins}분"
    df.loc[df['이름'] == name, '세전급여'] += gross_salary
    df.loc[df['이름'] == name, '4대보험'] += round(insurance, 2)
    df.loc[df['이름'] == name, '세후급여'] += round(gross_salary - insurance)
    df.loc[df['이름'] == name, '주휴수당'] += joohue

col = ['이름', '일한시간', '세전급여', '주휴수당', '4대보험', '세후급여']
df[col]

,이름,일한시간,세전급여,주휴수당,4대보험,세후급여
0,김정건,51시간 235분,471735.0,0.0,4541.90,467190
1,백정민,86시간 625분,828220.0,0.0,7974.10,820245
2,신수정,61시간 245분,559065.0,0.0,5382.65,553685
3,이웅,121시간 291분,1081056.0,0.0,10408.37,1070647
